Jupyter Notebooknotebook (unsaved changes)Python 3
Python 3 
File
Edit
View
Insert
Cell
Kernel
Help

Markdown
1. Scala's real-world project repository data
With almost 30k commits and a history spanning over ten years, Scala is a mature programming language. It is a general-purpose programming language that has recently become another prominent language for data scientists.

Scala is also an open source project. Open source projects have the advantage that their entire development histories -- who made changes, what was changed, code reviews, etc. -- are publicly available.

We're going to read in, clean up, and visualize the real world project repository of Scala that spans data from a version control system (Git) as well as a project hosting site (GitHub). We will find out who has had the most influence on its development and who are the experts.

The dataset we will use, which has been previously mined and extracted from GitHub, is comprised of three files:

pulls_2011-2013.csv contains the basic information about the pull requests, and spans from the end of 2011 up to (but not including) 2014.
pulls_2014-2018.csv contains identical information, and spans from 2014 up to 2018.
pull_files.csv contains the files that were modified by each pull request.
datasets/
# Importing pandas
import pandas as pd
# Loading in the data
pulls_one = pd.read_csv('datasets/pulls_2011-2013.csv')
pulls_two = pd.read_csv('datasets/pulls_2014-2018.csv')
pull_files = pd.read_csv('datasets/pull_files.csv') 
2. Preparing and cleaning the data
First, we will need to combine the data from the two separate pull DataFrames.

Next, the raw data extracted from GitHub contains dates in the ISO8601 format. However, pandas imports them as regular strings. To make our analysis easier, we need to convert the strings into Python's DateTime objects. DateTime objects have the important property that they can be compared and sorted.

The pull request times are all in UTC (also known as Coordinated Universal Time). The commit times, however, are in the local time of the author with time zone information (number of hours difference from UTC). To make comparisons easy, we should convert all times to UTC.

# Append pulls_one to pulls_two
pulls = pulls_one.append(pulls_two)
​
​
# Convert the date for the pulls object
pulls['date'] = pd.to_datetime(pulls['date'], utc = True)
3. Merging the DataFrames
The data extracted comes in two separate files. Merging the two DataFrames will make it easier for us to analyze the data in the future tasks.

pid
# Merge the two DataFrames
data = pulls.merge(pull_files, on = 'pid')
4. Is the project still actively maintained?
The activity in an open source project is not very consistent. Some projects might be active for many years after the initial release, while others can slowly taper out into oblivion. Before committing to contributing to a project, it is important to understand the state of the project. Is development going steadily, or is there a drop? Has the project been abandoned altogether?

The data used in this project was collected in January of 2018. We are interested in the evolution of the number of contributions up to that date.

For Scala, we will do this by plotting a chart of the project's activity. We will calculate the number of pull requests submitted each (calendar) month during the project's lifetime. We will then plot these numbers to see the trend of contributions.

A helpful reminder of how to access various components of a date can be found in this exercise of Data Manipulation with pandas

Additionally, recall that you can group by multiple variables by passing a list to groupby(). This video from Data Manipulation with pandas should help!

.plot(kind='bar', figsize = (12,4))
%matplotlib inline
​
# Create a column that will store the month
data['month'] = pd.DatetimeIndex(data['date']).month
​
# Create a column that will store the year
data['year'] = pd.DatetimeIndex(data['date']).year
​
# Group by the month and year and count the pull requests
counts = data.groupby(['month', 'year']).agg({'pid':'count'})
​
# Plot the results
counts.plot(kind='bar', figsize = (12,4))
<matplotlib.axes._subplots.AxesSubplot at 0x7f6540ebfa20>

5. Is there camaraderie in the project?
The organizational structure varies from one project to another, and it can influence your success as a contributor. A project that has a very small community might not be the best one to start working on. The small community might indicate a high barrier of entry. This can be caused by several factors, including a community that is reluctant to accept pull requests from "outsiders," that the code base is hard to work with, etc. However, a large community can serve as an indicator that the project is regularly accepting pull requests from new contributors. Such a project would be a good place to start.

In order to evaluate the dynamics of the community, we will plot a histogram of the number of pull requests submitted by each user. A distribution that shows that there are few people that only contribute a small number of pull requests can be used as in indicator that the project is not welcoming of new contributors.

# Required for matplotlib
%matplotlib inline

# Group by the submitter
by_user = data.groupby('user').agg({'uid':'count'})

# Plot the histogram
by_user.plot(kind='bar', figsize = (12,4))
# Required for matplotlib
%matplotlib inline
​
# Group by the submitter
by_user = data.groupby('user').agg({'uid':'count'})
​
# Plot the histogram
by_user.plot(kind='bar', figsize = (12,4))
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
/usr/local/lib/python3.5/dist-packages/pandas/core/indexes/base.py in get_loc(self, key, method, tolerance)
   2524             try:
-> 2525                 return self._engine.get_loc(key)
   2526             except KeyError:

pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc()

pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'uid'

During handling of the above exception, another exception occurred:

KeyError                                  Traceback (most recent call last)
<ipython-input-74-881354c64570> in <module>()
      3 
      4 # Group by the submitter
----> 5 by_user = data.groupby('user').agg({'uid':'count'})
      6 
      7 # Plot the histogram

/usr/local/lib/python3.5/dist-packages/pandas/core/groupby.py in aggregate(self, arg, *args, **kwargs)
   4289         versionadded=''))
   4290     def aggregate(self, arg, *args, **kwargs):
-> 4291         return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)
   4292 
   4293     agg = aggregate

/usr/local/lib/python3.5/dist-packages/pandas/core/groupby.py in aggregate(self, arg, *args, **kwargs)
   3722 
   3723         _level = kwargs.pop('_level', None)
-> 3724         result, how = self._aggregate(arg, _level=_level, *args, **kwargs)
   3725         if how is None:
   3726             return result

/usr/local/lib/python3.5/dist-packages/pandas/core/base.py in _aggregate(self, arg, *args, **kwargs)
    476 
    477                 try:
--> 478                     result = _agg(arg, _agg_1dim)
    479                 except SpecificationError:
    480 

/usr/local/lib/python3.5/dist-packages/pandas/core/base.py in _agg(arg, func)
    427                 result = compat.OrderedDict()
    428                 for fname, agg_how in compat.iteritems(arg):
--> 429                     result[fname] = func(fname, agg_how)
    430                 return result
    431 

/usr/local/lib/python3.5/dist-packages/pandas/core/base.py in _agg_1dim(name, how, subset)
    406                 aggregate a 1-dim with how
    407                 """
--> 408                 colg = self._gotitem(name, ndim=1, subset=subset)
    409                 if colg.ndim != 1:
    410                     raise SpecificationError("nested dictionary is ambiguous "

/usr/local/lib/python3.5/dist-packages/pandas/core/groupby.py in _gotitem(self, key, ndim, subset)
   4316         elif ndim == 1:
   4317             if subset is None:
-> 4318                 subset = self.obj[key]
   4319             return SeriesGroupBy(subset, selection=key,
   4320                                  grouper=self.grouper)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py in __getitem__(self, key)
   2137             return self._getitem_multilevel(key)
   2138         else:
-> 2139             return self._getitem_column(key)
   2140 
   2141     def _getitem_column(self, key):

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py in _getitem_column(self, key)
   2144         # get column
   2145         if self.columns.is_unique:
-> 2146             return self._get_item_cache(key)
   2147 
   2148         # duplicate columns & possible reduce dimensionality

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py in _get_item_cache(self, item)
   1840         res = cache.get(item)
   1841         if res is None:
-> 1842             values = self._data.get(item)
   1843             res = self._box_item_values(item, values)
   1844             cache[item] = res

/usr/local/lib/python3.5/dist-packages/pandas/core/internals.py in get(self, item, fastpath)
   3841 
   3842             if not isna(item):
-> 3843                 loc = self.items.get_loc(item)
   3844             else:
   3845                 indexer = np.arange(len(self.items))[isna(self.items)]

/usr/local/lib/python3.5/dist-packages/pandas/core/indexes/base.py in get_loc(self, key, method, tolerance)
   2525                 return self._engine.get_loc(key)
   2526             except KeyError:
-> 2527                 return self._engine.get_loc(self._maybe_cast_indexer(key))
   2528 
   2529         indexer = self.get_indexer([key], method=method, tolerance=tolerance)

pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc()

pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'uid'

6. What files were changed in the last ten pull requests?
Choosing the right place to make a contribution is as important as choosing the project to contribute to. Some parts of the code might be stable, some might be dead. Contributing there might not have the most impact. Therefore it is important to understand the parts of the system that have been recently changed. This allows us to pinpoint the "hot" areas of the code where most of the activity is happening. Focusing on those parts might not the most effective use of our times.

# Identify the last 10 pull requests
last_10 = ...
​
# Join the two data sets
joined_pr = ...
​
# Identify the unique files
files = ...
​
# Print the results
files
7. Who made the most pull requests to a given file?
When contributing to a project, we might need some guidance. We might find ourselves needing some information regarding the codebase. It is important direct any questions to the right person. Contributors to open source projects generally have other day jobs, so their time is limited. It is important to address our questions to the right people. One way to identify the right target for our inquiries is by using their contribution history.

We identified src/compiler/scala/reflect/reify/phases/Calculate.scala as being recently changed. We are interested in the top 3 developers who changed that file. Those developers are the ones most likely to have the best understanding of the code.

# This is the file we are interested in:
file = 'src/compiler/scala/reflect/reify/phases/Calculate.scala'
​
# Identify the commits that changed the file
file_pr = ...
​
# Count the number of changes made by each developer
author_counts = ...
​
# Print the top 3 developers
# ... YOUR CODE FOR TASK 7 ...
8. Who made the last ten pull requests on a given file?
Open source projects suffer from fluctuating membership. This makes the problem of finding the right person more challenging: the person has to be knowledgeable and still be involved in the project. A person that contributed a lot in the past might no longer be available (or willing) to help. To get a better understanding, we need to investigate the more recent history of that particular part of the system.

Like in the previous task, we will look at the history of src/compiler/scala/reflect/reify/phases/Calculate.scala.

file = 'src/compiler/scala/reflect/reify/phases/Calculate.scala'
​
# Select the pull requests that changed the target file
file_pr = ...
​
# Merge the obtained results with the pulls DataFrame
joined_pr = ...
​
# Find the users of the last 10 most recent pull requests
users_last_10 = ...
​
# Printing the results
users_last_10
9. The pull requests of two special developers
Now that we have identified two potential contacts in the projects, we need to find the person who was most involved in the project in recent times. That person is most likely to answer our questions. For each calendar year, we are interested in understanding the number of pull requests the authors submitted. This will give us a high-level image of their contribution trend to the project.

%matplotlib inline
​
# The developers we are interested in
authors = ['xeno-by', 'soc']
​
# Get all the developers' pull requests
by_author = ...
​
# Count the number of pull requests submitted each year
counts = by_author.groupby([..., ...]).agg({'pid': 'count'}).reset_index()
​
# Convert the table to a wide format
counts_wide = counts.pivot_table(index='date', columns='user', values='pid', fill_value=0)
​
# Plot the results
# ... YOUR CODE FOR TASK 9 ...
10. Visualizing the contributions of each developer
As mentioned before, it is important to make a distinction between the global expertise and contribution levels and the contribution levels at a more granular level (file, submodule, etc.) In our case, we want to see which of our two developers of interest have the most experience with the code in a given file. We will measure experience by the number of pull requests submitted that affect that file and how recent those pull requests were submitted.

authors = ['xeno-by', 'soc']
file = 'src/compiler/scala/reflect/reify/phases/Calculate.scala'
​
# Select the pull requests submitted by the authors, from the `data` DataFrame
by_author = ...
​
# Select the pull requests that affect the file
by_file = ...
​
# Group and count the number of PRs done by each user each year
grouped = by_file.groupby(['user', by_file['date'].dt.year]).count()['pid'].reset_index()
​
# Transform the data into a wide format
by_file_wide = ...
​
# Plot the results
by_file_wide.plot(kind='bar')
